<a href="https://colab.research.google.com/github/at9725/random-code/blob/main/%22SPY%22%2C_%22BND%22%2C%22GLD%22%2C%22QQQ%22%2C%22VTI%22_VAR_conf_level_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
!pip install matplotlib
!pip install scipy
import matplotlib.pyplot as plt
from scipy.stats import norm

In [ ]:
years =15
endDate = dt.datetime.now()

startDate = endDate - dt.timedelta(days = 365*years)




In [ ]:
tickers = ["SPY", "BND","GLD","QQQ","VTI"]


In [ ]:
adj_close_df = pd.DataFrame()

for ticker in tickers:
  data = yf.download(ticker, start=startDate, end = endDate)
  adj_close_df[ticker]= data["Adj Close"]

print(adj_close_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

                   SPY        BND         GLD         QQQ         VTI
Date                                                                 
2010-01-08   86.967712  51.390244  111.370003   40.659740   44.304878
2010-01-11   87.089188  51.409779  112.849998   40.493782   44.365944
2010-01-12   86.276962  51.605179  110.489998   39.987175   43.907925
2010-01-13   87.005646  51.487923  111.540001   40.485062   44.320133
2010-01-14   87.240967  51.611698  112.029999   40.519989   44.396473
...                ...        ...         ...         ...         ...
2024-12-27  595.010010  71.750000  241.399994  522.559998  294.070007
2024-12-30  588.219971  72.029999  240.630005  515.609985  290.820007
2024-12-31  586.080017  71.910004  242.130005  511.230011  289.809998
2025-01-02  584.640015  71.940002  245.419998  510.230011  289.260010
2025-01-03  591.950012  71.849998  243.490005  518.580017  293.059998

[3772 rows x 5 columns]


In [ ]:
log_returns = np.log(adj_close_df/adj_close_df.shift(1))
log_returns = log_returns.dropna()

print(log_returns)

                 SPY       BND       GLD       QQQ       VTI
Date                                                        
2010-01-11  0.001396  0.000380  0.013201 -0.004090  0.001377
2010-01-12 -0.009370  0.003794 -0.021134 -0.012590 -0.010377
2010-01-13  0.008410 -0.002275  0.009458  0.012374  0.009344
2010-01-14  0.002701  0.002401  0.004383  0.000862  0.001721
2010-01-15 -0.011288  0.000630 -0.010499 -0.011709 -0.011413
...              ...       ...       ...       ...       ...
2024-12-27 -0.010582 -0.002088 -0.006894 -0.013382 -0.010890
2024-12-30 -0.011477  0.003895 -0.003195 -0.013389 -0.011113
2024-12-31 -0.003645 -0.001667  0.006214 -0.008531 -0.003479
2025-01-02 -0.002460  0.000417  0.013496 -0.001958 -0.001900
2025-01-03  0.012426 -0.001252 -0.007895  0.016233  0.013051

[3771 rows x 5 columns]


In [ ]:
portfolio_value =  1000000
weights = np.array([1/len(tickers)]*len(tickers))
print(weights)

[0.2 0.2 0.2 0.2 0.2]


In [ ]:
historical_returns = (log_returns * weights).sum(axis = 1)
print(historical_returns)

Date
2010-01-11    0.002453
2010-01-12   -0.009936
2010-01-13    0.007462
2010-01-14    0.002414
2010-01-15   -0.008856
                ...   
2024-12-27   -0.008767
2024-12-30   -0.007056
2024-12-31   -0.002222
2025-01-02    0.001519
2025-01-03    0.006513
Length: 3771, dtype: float64


In [ ]:
days = 5

historical_x_day_returns = historical_returns.rolling(window = days).sum()

In [ ]:
cov_matrix = log_returns.cov()*252

In [ ]:
portfolio_std_dev = np.sqrt(weights.T @ cov_matrix @ weights)
print(portfolio_std_dev)

0.11572151141612182


In [ ]:
confidence_levels = [0.9, 0.95, 0.99]


In [ ]:
from scipy.stats import norm

VaRs = []

for cl in confidence_levels:
  VaR = portfolio_value * portfolio_std_dev * norm.ppf(cl) * np.sqrt(days/252)
  VaRs.append(VaR)

In [ ]:
print(f'{"Confidence Level": <20} {"Value at risk":<20}')
print('-'*40)

for cl, VaR in zip(confidence_levels, VaRs):
  print(f'{cl*100: > 6.0f}% {"":<8} ${VaR:>10,.2f}')

Confidence Level     Value at risk       
----------------------------------------
    90%          $ 20,889.83
    95%          $ 26,811.81
    99%          $ 37,920.45
